# ジョブマッチング 予測&生成AI デモ

In [1]:
import requests
import json

import pandas as pd
import numpy as np

from h2ogpte import H2OGPTE

In [2]:
MLOPS_ENDPOINT = 'https://model.internal.dedicated.h2o.ai/d4e5a4ac-9ca0-4e60-af0e-0b175c1997a8/model/score'

H2OGPTE_URL = 'https://h2ogpte.internal.dedicated.h2o.ai/'

with open('../_secret') as f:
    key = json.load(f)
H2OGPTE_KEY = key['global_internal-dedicated-0726']

In [40]:
!ls data

jd_db.csv                  resume_0106.docx
job_database.csv           resume_1303.docx
job_database_wCategory.csv resume_TaroShonan.docx
job_embeddings.csv


## データの抽出 - 生成AI
- 職務経歴書から、書かれているスキル（専門技術、マネジメント、経験等）を抽出する

参考: https://github.com/yukismd/H2O_LLM/blob/main/h2oGPTe/Global-API_RAG.ipynb

In [4]:
client = H2OGPTE(
    address='https://h2ogpte.internal.dedicated.h2o.ai',
    api_key=key['global_internal-dedicated-0726'],
)

client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.8`.
You can enable strict version checking by passing strict_version_check=True.


In [5]:
client.list_embedding_models()

['BAAI/bge-large-en-v1.5', 'BAAI/bge-m3', 'hkunlp/instructor-large', 'off']

In [23]:
# Create a new collection
collection_id = client.create_collection(
    name='skill-extractor',
    description='職務経歴書に対するRAG',
    embedding_model='BAAI/bge-m3',
    prompt_template_id='16095e26-6876-410f-a934-3c0651a21e4c'    # h2oGPTe UI（チャットの設定）から取得
)
collection_id

'264b76f7-d460-48a1-be75-c4f05c052060'

**検索を実施する職務経歴書**: [data/resume_0106.docx](data/resume_0106.docx)

In [24]:
# Upload documents
with open('data/resume_0106.docx', 'rb') as f:
    upload_file = client.upload('resume.docx', f)

upload_file

'4436cdaf-a81a-4c11-9776-35cdd5cc0317'

In [25]:
# Ingest documents (Creates previews, chunks and embeddings)
client.ingest_uploads(collection_id, [upload_file])

Job(id='4ba2cc92-11cc-49b6-9a4f-a25679ba378c', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 7, 29, 2, 17, 4, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='c4184dc7ee4847c0b02f20562af230aa', status='Collecting done.'), JobStatus(id='8e32806b56614437a80c4004ffc916b5', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 7, 29, 2, 17, 7, tzinfo=TzInfo(UTC)), duration='3s', duration_seconds=3.0)

In [26]:
# Collection内のDocument
client.list_documents_in_collection(collection_id, offset=0, limit=999)

[DocumentInfo(id='bbd1287d-a233-4929-a138-37232acfb8b7', username='yuki.shimada@h2o.ai', name='resume_TaroShonan.pdf', type='PDF', size=63547, page_count=2, pii_settings=None, connector='Upload', uri=None, original_type='Word', meta_data_dict={}, status='completed', updated_at=datetime.datetime(2024, 7, 29, 2, 17, 7, 750069, tzinfo=TzInfo(UTC)))]

In [27]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'meta-llama/Meta-Llama-3.1-70B-Instruct',
 'OpenGVLab/InternVL-Chat-V1-5',
 'h2oai/h2o-danube3-4b-chat',
 'mistral-medium-latest',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4o',
 'gemini-1.5-pro-latest']

In [28]:
# Create a chat session
chat_session_id = client.create_chat_session(collection_id)
chat_session_id

'85130503-1ce6-4d8c-8b47-221fdb8882aa'

In [29]:
Model = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

SysPrompt = '''
あなたは履歴書や職務経歴書から情報を取得するAIです。（回答は必ず日本語で実施します。）
'''

Message = '''
書かれているスキル（専門技術、マネジメント、経験等）を箇条描きで取り出して下さい。
'''


with client.connect(chat_session_id) as session:
    reply = session.query(
        message = Message,
        llm = Model,
        rag_config = {"rag_type": "rag"},   # (Normal) RAG 
        llm_args = dict(
            response_format = 'json_object',   # 出力はjsonを指定
            #guided_json = {'properties': {'skill_sets': {'type': 'string'}},}
        ),
    )
    skills = json.loads(reply.content)

print(skills)

{'skills': ['自動車販売', '自動車整備', '自動車保険の代理営業', '新車販売', '車検・点検', '自動車登録の書類作成', '指導員の業務指導や販売方法のアドバイス', '商談同席', 'Excel関数の使用やデータ表の作成', 'PowerPointの使用', '報告書、見積書、礼状などの文書作成', '会議資料、提案資料の作成', '普通自動車第一種運転免許', '損害保険募集人一般試験', 'TOEIC Listening & Reading Test 650点', '顧客とのコミュニケーション能力', 'Wordの使用']}


In [30]:
skills = [i for i in skills.values()][0]
skills

['自動車販売',
 '自動車整備',
 '自動車保険の代理営業',
 '新車販売',
 '車検・点検',
 '自動車登録の書類作成',
 '指導員の業務指導や販売方法のアドバイス',
 '商談同席',
 'Excel関数の使用やデータ表の作成',
 'PowerPointの使用',
 '報告書、見積書、礼状などの文書作成',
 '会議資料、提案資料の作成',
 '普通自動車第一種運転免許',
 '損害保険募集人一般試験',
 'TOEIC Listening & Reading Test 650点',
 '顧客とのコミュニケーション能力',
 'Wordの使用']

In [31]:
# Collectionの削除
client.delete_collections([collection_id])

Job(id='26b15759-b210-457a-9d69-5e120720344d', name='Deleting collections', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 7, 29, 2, 17, 28, tzinfo=TzInfo(UTC)), kind=<JobKind.DeleteCollectionsJob: 'DeleteCollectionsJob'>, statuses=[JobStatus(id='c4184dc7ee4847c0b02f20562af230aa', status='Deleting indexes...')], errors=[], last_update_date=datetime.datetime(2024, 7, 29, 2, 17, 29, tzinfo=TzInfo(UTC)), duration='1s', duration_seconds=1.0)

## 類似度検索 - 予測AI
- 抽出したスキル一覧とマッチする仕事をリストアップする

参考: https://github.com/yukismd/H2O_AI_Cloud/blob/main/sample_request/nlp_model/MLOps_HT_MetricLearning.ipynb

### スコアリング

In [32]:
# リクエスト用データ
json_data = dict(fields=["input"], rows=[[", ".join(skills)]])
json_data

{'fields': ['input'],
 'rows': [['自動車販売, 自動車整備, 自動車保険の代理営業, 新車販売, 車検・点検, 自動車登録の書類作成, 指導員の業務指導や販売方法のアドバイス, 商談同席, Excel関数の使用やデータ表の作成, PowerPointの使用, 報告書、見積書、礼状などの文書作成, 会議資料、提案資料の作成, 普通自動車第一種運転免許, 損害保険募集人一般試験, TOEIC Listening & Reading Test 650点, 顧客とのコミュニケーション能力, Wordの使用']]}

In [33]:
# MLOpsへのリクエスト
response = requests.post(url=MLOPS_ENDPOINT, 
                         headers={'content-type': 'application/json'}, 
                         data=json.dumps(json_data))
response

<Response [200]>

In [34]:
# 取得したEmbedding（256）次元
applicant_embedding = json.loads(response.json()['score'][0][0])['embeddings'][0]
len(applicant_embedding)
applicant_embedding[:10]

[-0.14998503029346466,
 -0.0906902402639389,
 0.31655681133270264,
 0.35003647208213806,
 -0.40125352144241333,
 0.40502312779426575,
 -0.09217745810747147,
 0.21565495431423187,
 -0.04981619492173195,
 -0.15057465434074402]

### 検索

In [41]:
# 検索対象の仕事リスト
df_jobdb = pd.read_csv('data/job_database.csv')
df_jobdb

,job_id,industry,role,rank,skill_sets
0,1,化学工業,安全管理者,シニア,- 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全...
1,2,物流・運輸業,物流コーディネーター,ミドル,- 物流業界の知識と経験\n- 日本語と英語の流暢なコミュニケーション能力\n- 計画と調整...
2,3,農業,物流担当,ミドル,- 農業物流の知識と経験\n- 貨物管理と配送計画の作成\n- 仕入れと倉庫管理のスキル\n...
3,4,物流・運輸業,倉庫管理,ジュニア,- 倉庫管理の基礎知識\n- 在庫管理の基本的な手続き\n- 品質管理の知識\n- 安全管理...
4,5,卸売業,受注管理,ジュニア,- 注文受付と確認の手続きに堪能な経験\n- 在庫管理と注文処理の知識\n- 商品情報の入力...
...,...,...,...,...,...
495,496,メディア・エンターテインメント業,記者,ミドル,- 5年以上の報道経験\n- 新聞・テレビ・ウェブメディアなど、異なるメディアプラットフォー...
496,497,小売業,店舗マネージャー,ミドル,- 店舗管理経験: ミドルの店舗マネージャーとして働くには、少なくとも1~2年以上の店舗管理...
497,498,リサイクル業,技術サポート,ジュニア,- リサイクル業界の知識\n- 顧客サポート経験\n- トラブルシューティング能力\n- 英...
498,499,広告・マーケティング業,マーケティングアナリスト,ジュニア,- データ分析スキル：広告キャンペーンやウェブサイトのアクセスログなどのデータを分析する能力...


In [36]:
# 検索対象の仕事(Job ID)とEmbedding。事前に作成済み（Hydrogen Torch上でのスコアリング結果を保存）
df_job_embeddings = pd.read_csv('data/job_embeddings.csv')
df_job_embeddings_only = df_job_embeddings.iloc[:, 1:]
df_job_embeddings_only.shape

(500, 256)

In [38]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def search_top_similarity(applicant_embedding :list) -> pd.DataFrame:
    similarity_score = [cos_sim(applicant_embedding, df_job_embeddings_only.iloc[i,:]) for i in range(df_job_embeddings_only.shape[0])]
    df_similarity_score = pd.DataFrame({'job_id':df_job_embeddings['job_id'], 'similarity':similarity_score})
    df_similarity_score = df_similarity_score.sort_values('similarity', ascending=False)
    df_similar_top10 = df_similarity_score[:10]
    df_similar_top10 = pd.merge(df_similar_top10, df_jobdb, on='job_id', how='left')
    return df_similar_top10

In [39]:
# 入力したスキルセットにマッチする、上位の仕事
search_top_similarity(applicant_embedding)

,job_id,similarity,industry,role,rank,skill_sets
0,360,0.072274,広告・マーケティング業,プランナー,シニア,- 5年以上のプランニング経験\n- 広告・マーケティング戦略の立案能力\n- データ分析の...
1,401,0.071788,卸売業,カスタマーサポート,ミドル,- 顧客サポート経験: ミドルのカスタマーサポートとしては、顧客サポートに関する経験が必要で...
2,365,0.071461,リサイクル業,品質管理,シニア,- 厳格な品質標準を実装する能力\n- 品質管理システムの設計と管理\n- 産業標準（ISO...
3,404,0.070987,メディア・エンターテインメント業,放送技術者,ミドル,- 放送技術の基礎知識と実務能力\n- 録音、編集、配信技術の知識と経験\n- デジタルテレ...
4,304,0.070539,ITサービス業,カスタマーサポート,ジュニア,- 顧客サポート経験: 少ない程度の顧客サポート経験があること。\n- コミュニケーション能...
5,5,0.069380,卸売業,受注管理,ジュニア,- 注文受付と確認の手続きに堪能な経験\n- 在庫管理と注文処理の知識\n- 商品情報の入力...
6,16,0.068283,自動車製造業,製造オペレーター,ミドル,- 複雑な機械操作能力：製造オペレーターとしては、複雑な機械を操作する能力が必要です。\n-...
7,15,0.068118,医療・ヘルスケア業,医療技師,ジュニア,- 医療法令・ガイドラインに従う能力\n- 医療機器の操作能力\n- 患者のケアや治療に堪能...
8,6,0.067742,広告・マーケティング業,グラフィックデザイナー,シニア,- 高度なグラフィックデザイン能力\n- 5年以上の経験\n- 広告・マーケティング戦略に基...
9,180,0.067658,不動産業,顧客サービス担当,ミドル,- 不動産業の知識と経験\n- 顧客サポートに堪能なコミュニケーション能力\n- 不動産営業...
